<a href="https://colab.research.google.com/github/dradra751273/gitDemo/blob/main/Convert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive/forLily/*.py
import sys
sys.path.append('/content/drive/MyDrive/forLily')
import pandas as pd
from constants import oriCols, meetingCols, units, jobTitle_dict, colsWidth, colsDonotMerge, fieldsName
from reformatFuncs import reorganize_data, meeting_format

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/forLily/constants.py
/content/drive/MyDrive/forLily/reformatFuncs.py


請設定檔名

In [2]:
csvFile = '/content/drive/MyDrive/forLily/OriSource_1.csv'
df = pd.read_csv(csvFile)
df = reorganize_data(df, oriCols)
df = meeting_format(df, meetingCols, units, jobTitle_dict)
df

,單位,職稱,姓名,身分證號,官職等,俸級,俸點,獎懲_嘉獎,獎懲_記功,獎懲_記大功,...,考績_1,考績_2,考績_3,考績_4,考績_5,單位主管評擬分數,考績會初核分數,機關首長分數,考列甲等適用條款,備考
1,署長室,專員,李承穎,F124804248,警正一階,年功俸一級,625,31,8,,...,甲,甲,甲,甲,甲,,,,特殊條件 3 目\n一般條件 目,
2,署長室,警務正,張嘉玲,C221126693,警正二階,年功俸一級,500,27,5,1,...,甲,甲,甲,甲,甲,,,,特殊條件 3 目\n一般條件 目,
3,副署長室,副署長,劉柏良,K120090537,警監一階,年功俸一級,770,7,8,,...,甲,甲,甲,甲,甲,,,,特殊條件 3 目\n一般條件 目,
4,副署長室,副署長,陳永利,Q121296279,警監一階,年功俸一級,770,10,,,...,甲,甲,甲,甲,甲,,,,特殊條件 3 目\n一般條件 目,
5,副署長室,副署長,林順家,P121330748,警監一階,年功俸一級,770,6,10,,...,甲,甲,甲,甲,甲,,,,特殊條件 3 目\n一般條件 目,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,法制室,警務正,劉書伶,E221249728,警正二階,年功俸一級,575,22,,,...,甲,甲,乙,甲,甲,,,,特殊條件 3 目\n一般條件 目,
471,法制室,警務正,林宜擇,T122479060,警正二階,年功俸一級,575,29,1,,...,甲,甲,甲,甲,甲,,,,特殊條件 3 目\n一般條件 目,
472,法制室,警務正,趙庭輝,R122977407,警正二階,年功俸一級,575,54,3,,...,甲,甲,甲,乙,甲,,,,特殊條件 3 目\n一般條件 目,
473,法制室,警務正,吳偉豪,Y120524181,警正二階,年功俸一級,575,16,2,,...,甲,甲,甲,甲,乙,,,,特殊條件 3 目\n一般條件 目,


In [3]:
import openpyxl as pyxl
from openpyxl.styles import Alignment, Font, Border, Side


def merge_cells(excelFile, colsDonotMerge):
    wb = pyxl.load_workbook(excelFile)
    ws = wb.active
    # title
    ws.merge_cells('A1:Y1')
    #  data field 
    cols = [chr(i).upper() for i in range(ord('a'), ord('z'))]
    for col in cols:
        if col not in colsDonotMerge:
            ws.merge_cells(f'{col}2:{col}3')

    ws.merge_cells('I2:N2')
    ws.merge_cells('R2:T2')
    wb.save(excelFile)


def fields_name(excelFile, fieldsName):
    wb = pyxl.load_workbook(excelFile)
    ws = wb.active
    for field, name in fieldsName.items():
        ws[field] = name   
    wb.save(excelFile)


def populate_data(excelFile, df):
    wb = pyxl.load_workbook(excelFile)
    ws = wb.active
    for iRow in range(df.shape[0]):
        ws.cell(row=iRow + 4, column=1).value = iRow + 1
        for iCol in range(df.shape[1]):
            ws.cell(row=iRow + 4, column=iCol + 2).value = df.iloc[iRow, iCol]
    wb.save(excelFile)
    

def align_elements(excelFile):
    wb = pyxl.load_workbook(excelFile)
    ws = wb.active
    for row in ws[f'A1:Y{ws.max_row}']:
        for cell in row:
            cell.alignment = Alignment(
                horizontal='center', vertical='center'
                )
    wb.save(excelFile)

def style_font(excelFile):
    wb = pyxl.load_workbook(excelFile)
    ws = wb.active
    for row in ws[f'A1:Y{ws.max_row}']:
        for cell in row:
            cell.font = Font(name='DFKai-SB', size=13)
    # rewards
    for col in ['I', 'J', 'K', 'L', 'M', 'N']:
        ws[f'{col}3'].font = Font(name='DFKai-SB', size=10)
    ws['R2'].font = Font(name='DFKai-SB', size=12)
    # records
    for col in ['U', 'V', 'W']:
        ws[f'{col}2'].font = Font(name='DFKai-SB', size=12)
    # title
    ws['A1'].font = Font(name='DFKai-SB', size=22, bold=True)
    wb.save(excelFile)


def adjust_column_width(excelFile, cols_width):
    wb = pyxl.load_workbook(excelFile)
    ws = wb.active
    for col, width in cols_width.items():
      ws.column_dimensions[col].width = width
    wb.save(excelFile)


def set_border(excelFile):
    wb = pyxl.load_workbook(excelFile)
    ws = wb.active
    thin_border = Border(
        left=Side(style='thin'), right=Side(style='thin'), 
        top=Side(style='thin'), bottom=Side(style='thin')
        )
    for row in ws[f'A2:Y{ws.max_row}']:
        for cell in row:
            cell.border = thin_border
    wb.save(excelFile)

def adjust_row_height(excelFile):
    wb = pyxl.load_workbook(excelFile)
    ws = wb.active
    ws.row_dimensions[1].height = 55
    for row in range(ws.max_row - 3):
        ws.row_dimensions[row + 4].height = 45
    wb.save(excelFile)
    

In [4]:
merge_cells('test.xlsx', colsDonotMerge)
fields_name('test.xlsx', fieldsName)
populate_data('test.xlsx', df)
align_elements('test.xlsx')
style_font('test.xlsx')
set_border('test.xlsx')
adjust_row_height('test.xlsx')
adjust_column_width('test.xlsx', colsWidth)